### Saving time-averages of the data in separate files

In [20]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from xgcm import Grid
import filter
from dask.diagnostics import ProgressBar
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

from dask.distributed import Client
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = Client(cluster)

client

Client Scheduler: tcp://127.0.0.1:41991 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 16 Memory: 406.01 GB


In [21]:
get_var = ['hf_CAu_2d', 'hf_CAv_2d', 'hf_PFu_2d', 'hf_PFv_2d', 'hf_diffu_2d', 'hf_diffv_2d', 'hf_du_dt_visc_2d', 
            'hf_dudt_2d', 'hf_dv_dt_visc_2d', 'hf_dvdt_2d', 'hf_gKEu_2d', 'hf_gKEv_2d', 'hf_rvxu_2d',
            'hf_rvxv_2d', 'hf_u_BT_accel_2d', 'hf_v_BT_accel_2d','pbo','taux', 'taux_bot', 'tauy', 'tauy_bot',
            'ubt_dt','vbt_dt','wfo','col_height','intz_CAu_2d','intz_CAv_2d','intz_diffu_2d','intz_diffv_2d',
           'intz_gKEu_2d', 'intz_gKEv_2d', 'intz_PFu_2d', 'intz_PFv_2d', 'intz_rvxu_2d', 'intz_rvxv_2d',
          'intz_u_BT_accel_2d', 'intz_v_BT_accel_2d']

# Note - intz variables are only available in v2 runs

#### Read Data first (for OM125 output)

In [22]:
#path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/"
path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v2/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/"

ds_full = xr.open_mfdataset(path + "ocean_monthly.000101-000512*.nc")


ds_full = ds_full.get(get_var)

#ppdir1 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"
ppdir1 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v2/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"

ds_grid = xr.open_dataset(ppdir1 + "ocean_monthly.static.nc")

ds_full = ds_full.isel(xq = slice(1,2881), yq=slice(1,2241))
ds_grid = ds_grid.isel(xq = slice(0,2880), yq=slice(0,2240))

ds_full = xr.merge([ds_full, ds_grid])

In [23]:
print(ds_full)

<xarray.Dataset>
Dimensions:             (time: 60, xh: 2880, xq: 2880, yh: 2240, yq: 2240)
Coordinates:
  * time                (time) object 0001-01-16 12:00:00 ... 0005-12-16 12:0...
  * xh                  (xh) float64 -298.6 -298.5 -298.4 ... 61.05 61.16 61.28
  * yh                  (yh) float64 -83.62 -83.58 -83.54 ... 89.86 89.92 89.97
  * xq                  (xq) float64 -298.5 -298.4 -298.3 ... 61.12 61.23 61.35
  * yq                  (yq) float64 -83.6 -83.56 -83.52 ... 89.89 89.95 90.0
Data variables:
    hf_CAu_2d           (time, yh, xq) float32 dask.array<chunksize=(60, 2240, 2880), meta=np.ndarray>
    hf_CAv_2d           (time, yq, xh) float32 dask.array<chunksize=(60, 2240, 2880), meta=np.ndarray>
    hf_PFu_2d           (time, yh, xq) float32 dask.array<chunksize=(60, 2240, 2880), meta=np.ndarray>
    hf_PFv_2d           (time, yq, xh) float32 dask.array<chunksize=(60, 2240, 2880), meta=np.ndarray>
    hf_diffu_2d         (time, yh, xq) float32 dask.array<chunksize=

In [24]:
#path2 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c192_OM4p125/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/"
path2 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v2/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/"

ds_vmo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.vmo.nc")
ds_umo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.umo.nc")

In [25]:
%%time

#ds_save = (ds_full.chunk({'xh': 500, 'yh': 500, 'xq': 500, 'yq': 500})).mean('time')
ds_save = (ds_full).mean('time')

vmo_2d = (ds_vmo['vmo'].isel(yq=slice(1,2241),time=slice(0,5)).mean('time')).sum('z_l')

umo_2d = (ds_umo['umo'].isel(xq=slice(1,2881),time=slice(0,5)).mean('time')).sum('z_l')

ds_save = xr.merge([ds_save, vmo_2d, umo_2d])

CPU times: user 50.3 s, sys: 22.6 s, total: 1min 12s
Wall time: 1min 10s


In [26]:
print(ds_save)

<xarray.Dataset>
Dimensions:             (xh: 2880, xq: 2880, yh: 2240, yq: 2240)
Coordinates:
  * xh                  (xh) float64 -298.6 -298.5 -298.4 ... 61.05 61.16 61.28
  * yh                  (yh) float64 -83.62 -83.58 -83.54 ... 89.86 89.92 89.97
  * xq                  (xq) float64 -298.5 -298.4 -298.3 ... 61.12 61.23 61.35
  * yq                  (yq) float64 -83.6 -83.56 -83.52 ... 89.89 89.95 90.0
Data variables:
    hf_CAu_2d           (yh, xq) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_CAv_2d           (yq, xh) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_PFu_2d           (yh, xq) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_PFv_2d           (yq, xh) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_diffu_2d         (yh, xq) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_diffv_2d         (yq, xh) float32 dask.array<chunksize=(2240, 2880), meta=np.ndarray>
    hf_du_dt_

In [27]:
# for OM125
%time ds_save = ds_save.load()
%time ds_save.to_netcdf("/work/Hemant.Khatri/MOM6_idealised/MOM_budget/Global_MOM6/v2/Mom_Budget_OM125_0001_0005.nc")

CPU times: user 6.64 s, sys: 1.98 s, total: 8.62 s
Wall time: 1min 27s
CPU times: user 327 ms, sys: 1.34 s, total: 1.66 s
Wall time: 1.88 s


#### Read Data first (for OM25 output)

In [3]:
%%time

path = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c96_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/ts/monthly/5yr/"

ds_full = xr.open_mfdataset(path + "ocean_monthly.000101-000512*.nc", parallel=True, decode_times=False)
ds_full = ds_full.isel(xq = slice(1,1441), yq=slice(1,1081))

ds_full = ds_full.get(get_var)

#ppdir1 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c96_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly/"
ppdir1 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20210308/CM4_piControl_c192_OM4p125_v2/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_daily_d2/"

ds_grid = xr.open_dataset(ppdir1 + "ocean_daily_d2.static.nc") #"ocean_monthly.static.nc")
ds_grid = ds_grid.isel(xq = slice(1,1441), yq=slice(1,1081))

#ds_full = xr.merge([ds_full, ds_grid])

CPU times: user 8.09 s, sys: 283 ms, total: 8.38 s
Wall time: 18.1 s


In [4]:
print(ds_grid)

<xarray.Dataset>
Dimensions:       (xh: 1440, xq: 1440, yh: 1120, yq: 1080)
Coordinates:
  * xh            (xh) float64 -298.4 -298.2 -298.0 -297.7 ... 60.89 61.12 61.35
  * xq            (xq) float64 -298.3 -298.1 -297.8 -297.6 ... 61.0 61.23 61.35
  * yh            (yh) float64 -83.56 -83.48 -83.41 -83.33 ... 89.78 89.89 90.0
  * yq            (yq) float64 -83.52 -83.45 -83.37 -83.29 ... 85.45 85.56 85.67
Data variables:
    Coriolis      (yq, xq) float32 ...
    areacello     (yh, xh) float32 ...
    areacello_bu  (yq, xq) float32 ...
    areacello_cu  (yh, xq) float32 ...
    areacello_cv  (yq, xh) float32 ...
    basin         (yh, xh) float64 ...
    deptho        (yh, xh) float64 ...
    dxCu          (yh, xq) float32 ...
    dxCv          (yq, xh) float32 ...
    dxt           (yh, xh) float32 ...
    dyCu          (yh, xq) float32 ...
    dyCv          (yq, xh) float32 ...
    dyt           (yh, xh) float32 ...
    geolat        (yh, xh) float32 ...
    geolat_c      (yq, xq) 

In [4]:
%%time

path2 = "/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20201120/CM4_piControl_c96_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_annual_z/ts/annual/10yr/"

ds_vmo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.vmo.nc")
ds_vmo = ds_vmo.isel(yq=slice(1,1081),time=slice(0,5)).chunk({'xh': 500,'yq': 500})

ds_umo = xr.open_dataset(path2 + "ocean_annual_z.0001-0010.umo.nc")
ds_umo = ds_umo.isel(xq = slice(1,1441),time=slice(0,5)).chunk({'xq': 500,'yh': 500})

CPU times: user 43 ms, sys: 14 ms, total: 57 ms
Wall time: 413 ms


In [5]:
# Time-mean
%time ds_full_mean = (ds_full.chunk({'xh': 500, 'yh': 500, 'xq': 500, 'yq': 500})).mean('time').load()

%time umo_2d = (ds_umo.mean('time')).sum('z_l').drop(['z_i']).load()

%time vmo_2d = (ds_vmo.mean('time')).sum('z_l').drop(['z_i']).load()

CPU times: user 1.67 s, sys: 388 ms, total: 2.05 s
Wall time: 16.7 s
CPU times: user 371 ms, sys: 40 ms, total: 411 ms
Wall time: 8.53 s
CPU times: user 413 ms, sys: 48 ms, total: 461 ms
Wall time: 9.29 s


In [8]:
print(ds_save)

<xarray.Dataset>
Dimensions:           (nv: 2, xh: 1440, xq: 1440, yh: 1080, yq: 1080)
Coordinates:
  * xh                (xh) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * yh                (yh) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * xq                (xq) float64 -299.6 -299.3 -299.1 ... 59.66 59.91 60.16
  * yq                (yq) float64 -80.35 -80.27 -80.19 ... 89.78 89.89 90.0
  * nv                (nv) float64 1.0 2.0
Data variables:
    hf_CAu_2d         (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_CAv_2d         (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_PFu_2d         (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_PFv_2d         (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_diffu_2d       (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_diffv_2d       (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_du_dt_visc_2d  (yh, xq) float32 nan nan nan nan n

In [7]:
%%time
ds_save = (xr.merge([ds_full_mean, umo_2d, vmo_2d, ds_grid]))

CPU times: user 10 ms, sys: 1 ms, total: 11 ms
Wall time: 10 ms


In [10]:
#%%time
print(ds_save)

# for OM125
#ds_save.to_netcdf("/work/Hemant.Khatri/MOM6_idealised/MOM_budget/Global_MOM6/Mom_Budget_OM125_0001_0005.nc")

# for OM25
%time ds_save.to_netcdf("/work/Hemant.Khatri/MOM6_idealised/MOM_budget/Global_MOM6/Mom_Budget_OM25_0001_0005.nc")

# Note: Do not use hyphen in nc file names. It slows down file writing badly. 

<xarray.Dataset>
Dimensions:           (nv: 2, xh: 1440, xq: 1440, yh: 1080, yq: 1080)
Coordinates:
  * xh                (xh) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * yh                (yh) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * xq                (xq) float64 -299.6 -299.3 -299.1 ... 59.66 59.91 60.16
  * yq                (yq) float64 -80.35 -80.27 -80.19 ... 89.78 89.89 90.0
  * nv                (nv) float64 1.0 2.0
Data variables:
    hf_CAu_2d         (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_CAv_2d         (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_PFu_2d         (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_PFv_2d         (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_diffu_2d       (yh, xq) float32 nan nan nan nan nan ... nan nan nan nan
    hf_diffv_2d       (yq, xh) float32 nan nan nan nan nan ... nan nan nan nan
    hf_du_dt_visc_2d  (yh, xq) float32 nan nan nan nan n

In [18]:
drop_var = ['Heat_PmE', 'LwLatSens', 'col_mass', 'evs', 'ficeberg', 'friver','fsitherm','heat_content_cond', 
           'heat_content_fprec','heat_content_massin', 'heat_content_massout', 'heat_content_surfwater',
           'hfds', 'hfevapds', 'hfibthermds', 'hflso', 'hfrainds', 'hfrunoffds', 'hfsifrazil', 'hfsnthermds', 
           'hfsso', 'mass_wt', 'net_heat_coupler', 'net_massin', 'net_massout', 'nonpenSW', 'opottempmint',
          'prlq', 'prsn', 'rlntds', 'rsdo', 'rsntds', 'sfdsi', 'somint', 'tauuo', 'tauvo', 'ustar']

ds_save = ds_full.drop(drop_var)

ds_save = ds_save.mean('time')

distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [7]:
print(list(ds_full.keys()))

['Heat_PmE', 'average_DT', 'average_T1', 'average_T2', 'time_bnds', 'LwLatSens', 'MLD_003', 'MLD_003_max', 'MLD_003_min', 'MLD_EN1', 'MLD_EN1_max', 'MLD_EN1_min', 'MLD_EN2', 'MLD_EN3', 'S_adx_2d', 'S_ady_2d', 'T_adx_2d', 'T_ady_2d', 'col_height', 'col_mass', 'ePBL_h_ML', 'evs', 'ficeberg', 'friver', 'fsitherm', 'heat_content_cond', 'heat_content_fprec', 'heat_content_massin', 'heat_content_massout', 'heat_content_surfwater', 'hf_CAu_2d', 'hf_CAv_2d', 'hf_PFu_2d', 'hf_PFv_2d', 'hf_diffu_2d', 'hf_diffv_2d', 'hf_du_dt_visc_2d', 'hf_dudt_2d', 'hf_dv_dt_visc_2d', 'hf_dvdt_2d', 'hf_gKEu_2d', 'hf_gKEv_2d', 'hf_rvxu_2d', 'hf_rvxv_2d', 'hf_u_BT_accel_2d', 'hf_v_BT_accel_2d', 'hfds', 'hfevapds', 'hfibthermds', 'hflso', 'hfrainds', 'hfrunoffds', 'hfsifrazil', 'hfsnthermds', 'hfsso', 'mass_wt', 'net_heat_coupler', 'net_massin', 'net_massout', 'nonpenSW', 'opottempmint', 'pbo', 'prlq', 'prsn', 'rlntds', 'rsdo', 'rsntds', 'sfdsi', 'sob', 'somint', 'tauuo', 'tauvo', 'taux', 'taux_bot', 'tauy', 'tauy_

In [ ]:
#get_var = ['hf_CAu_2d', 'hf_CAv_2d', 'hf_PFu_2d', 'hf_PFv_2d', 'hf_diffu_2d', 'hf_diffv_2d', 'hf_du_dt_visc_2d', 
#            'hf_dudt_2d', 'hf_dv_dt_visc_2d', 'hf_dvdt_2d', 'hf_gKEu_2d', 'hf_gKEv_2d', 'hf_rvxu_2d',
#            'hf_rvxv_2d', 'hf_u_BT_accel_2d', 'hf_v_BT_accel_2d','pbo','taux', 'taux_bot', 'tauy', 'tauy_bot',
#            'ubt_dt','vbt_dt','wfo','col_height']

In [35]:
print(ds_save.nbytes/1e9)

0.329743032


In [28]:
ds_save.close()
ds_umo.close()
ds_vmo.close()
ds_full.close()
client.close()
cluster.close()